In [ ]:
######### NOVO CÓDIGO 333333333333333

In [ ]:
import pandas as pd
from datetime import date, timedelta, datetime
import re

# Define a data de ontem
hoje = date.today()
#ontem = date(2025, 10, 21)

print(f"Filtrando registros para: {hoje.strftime('%d/%m/%Y')}")


In [ ]:
arquivos = ['Ponto_Algodoeira.txt', 'Ponto_Escritorio.txt', 'Ponto_Sede.txt', 'Ponto_Secador.txt']
dfs_ponto = []

for arquivo in arquivos:
    df = pd.read_csv(arquivo, header=None, names=['linha_completa'], encoding='latin-1')
    df['sequencia'] = df['linha_completa'].str.slice(0, 10)
    df['data'] = pd.to_datetime(df['linha_completa'].str.slice(10, 18), format='%d%m%Y', errors='coerce')
    df['hora'] = pd.to_datetime(df['linha_completa'].str.slice(18, 22), format='%H%M', errors='coerce').dt.time
    df['data_hora'] = pd.to_datetime(
        df['data'].dt.strftime('%d/%m/%Y') + ' ' + df['hora'].astype(str),
        format='%d/%m/%Y %H:%M',
        errors='coerce'
    )
    df['NIT'] = df['linha_completa'].apply(lambda linha: re.findall(r'\d{11}', linha)[-1] if re.findall(r'\d{11}', linha) else None)
    dfs_ponto.append(df)
    df['origem'] = arquivo  # Marca de qual arquivo veio cada linha

# Junta todos os arquivos processados
df_ponto = pd.concat(dfs_ponto, ignore_index=True)



In [ ]:
# Fixar a data
data_alvo = hoje

# Filtrar registros da data
df_dia = df_ponto[df_ponto['data'].dt.date == data_alvo]

# Contar quantos registros cada NIT teve
contagem = df_dia['NIT'].value_counts()

# Filtrar apenas NITs com um único registro
nits_unicos = contagem[contagem == 1].index

# Salvar em arquivo .txt
nome_arquivo_nits = 'nits_unicos.txt'

try:
    with open(nome_arquivo_nits, 'w', encoding='utf-8') as f:
        f.write(f"NITs com ponto único em {data_alvo.strftime('%d/%m/%Y')}\n")
        f.write("=" * 40 + "\n\n")
        for nit in nits_unicos:
            f.write(f"{nit}\n")
    print(f"\n✓ Arquivo gerado com sucesso: {nome_arquivo_nits}")
except Exception as e:
    print(f"\n✗ Erro ao salvar arquivo: {e}")


In [ ]:
# Fixar a data
data_alvo = hoje

# Filtrar registros da data
df_dia = df_ponto[df_ponto['data'].dt.date == data_alvo]

# Contar quantos registros cada NIT teve
contagem = df_dia['NIT'].value_counts()

# Filtrar apenas NITs com um único registro
nits_unicos = contagem[contagem == 1].index

# Filtrar DataFrame com esses NITs
df_unico = df_dia[df_dia['NIT'].isin(nits_unicos)]

# Contar por origem
resultado = df_unico['origem'].value_counts()
print(resultado)
